# Introduction

This notebook uses tranfer learning to classify spectrogram data on dolphin whistles. Much of the code used here was adpapted from https://keras.io/guides/transfer_learning/.

Credit for parts of the code dealing with the `ImageDataGenerator` is given to Josh Wheeler and Gemma Ruseva (https://github.com/JoshWheeler08/DolphinAcoustics-Classifier/blob/main/vip_dolphin.ipynb)

The code seeks to make a model which can distinguish among Common, Melon Head, and Bottlenose dolphin species.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
from tensorflow.keras import layers

# TODO: Ensure data is standardized properly before training (look especially at rescaling code)
# TODO: fine tune on different spectrogram data used to fit model

In [ ]:
# Mount Google Drive so data can be accessed
from google.colab import drive
drive.mount('/content/drive') # '/content' is the current working directory

Mounted at /content/drive


# Loading the Base Model and Adding Extra Layers

Here we load the Xception base model for transfer learning.This is a fairly complex CNN-based model. More can be read about it here: https://arxiv.org/abs/1610.02357.

The base model is "frozen" so that its hyperparameters are not drastically changed by subsequent training. 

We then add two new layers for training on the spectrogram data.

A list of several alternatives to the Xception base model can be found here: https://keras.io/api/applications/#available-models. 

In [ ]:
# input shape set to that used with the dolphins spectrogram data
image_shape = (413, 202, 3)

base_model = keras.applications.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=image_shape, 
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=image_shape)  # (150, 150, 3)
x = keras.Sequential()(inputs)#data_augmentation(inputs)

# Pre-trained Xception weights requires that input be scaled
# from (0, 255) to a range of (-1., +1.), the rescaling layer
# outputs: `(inputs * scale) + offset`
scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(x)

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout

number_of_outputs = 3 # Here, the number of outputs is set to three since we are working 
                      # on a three-category multi-classification problem.
outputs = keras.layers.Dense(number_of_outputs)(x)
model = keras.Model(inputs, outputs)

model.summary()

83697664/83683744 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 413, 202, 3)]     0         
                                                                 
 sequential (Sequential)     multiple                  0         
                                                                 
 rescaling (Rescaling)       (None, 413, 202, 3)       0         
                                                                 
 xception (Functional)       (None, 13, 7, 2048)       20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0      

# Loading the Training, Test and Validation Data

The spectrogram image data are now loaded.

In [ ]:
# code section adapted from https://github.com/JoshWheeler08/DolphinAcoustics-Classifier/blob/main/vip_dolphin.ipynb

IMAGE_SHAPE = (413, 202)
directory_name = "/content/drive/MyDrive/Dolphin_Acoustics_VIP/model_test_train/"

TEST_DATA_DIR = directory_name + "test"
TRAINING_DATA_DIR = directory_name + "train"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=.20
) #https://stackoverflow.com/questions/42443936/keras-split-train-test-set-when-using-imagedatagenerator

test_generator = ImageDataGenerator(
                    rescale=1./255
                ).flow_from_directory(
                      TEST_DATA_DIR,
                      shuffle=True,
                      batch_size = 50,
                      target_size=IMAGE_SHAPE
                      )
validation_generator = train_datagen.flow_from_directory(
    TRAINING_DATA_DIR,
    subset="validation",
    shuffle=True,
    target_size=IMAGE_SHAPE
)
                
train_generator = train_datagen.flow_from_directory(
    TRAINING_DATA_DIR,
    subset="training",
    shuffle=True,
    target_size=IMAGE_SHAPE
)

Found 600 images belonging to 3 classes.
Found 480 images belonging to 3 classes.
Found 1920 images belonging to 3 classes.


# Fitting the Model to the Spectrogram image data.

We now compile and fit the model. The new classification layers are trained, while the base model's hyperparameters remain unchanged.

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.CategoricalAccuracy()]
)

epochs = 10
#image_data, image_labels = test_generator.next()
model.fit(test_generator, epochs=epochs, validation_data=validation_generator)
model.save(directory_name + "transfer_learning_classifier.model")

Epoch 1/10
12/12 [==============================] - 237s 19s/step - loss: 1.1059 - categorical_accuracy: 0.3333 - val_loss: 1.0966 - val_categorical_accuracy: 0.3333
Epoch 2/10
12/12 [==============================] - 21s 2s/step - loss: 1.1098 - categorical_accuracy: 0.3067 - val_loss: 1.0966 - val_categorical_accuracy: 0.3333
Epoch 3/10
12/12 [==============================] - 21s 2s/step - loss: 1.0970 - categorical_accuracy: 0.3717 - val_loss: 1.0936 - val_categorical_accuracy: 0.5646
Epoch 4/10
12/12 [==============================] - 21s 2s/step - loss: 1.1011 - categorical_accuracy: 0.3383 - val_loss: 1.0936 - val_categorical_accuracy: 0.3333
Epoch 5/10
12/12 [==============================] - 21s 2s/step - loss: 1.0910 - categorical_accuracy: 0.3900 - val_loss: 1.0908 - val_categorical_accuracy: 0.5437
Epoch 6/10
12/12 [==============================] - 21s 2s/step - loss: 1.0953 - categorical_accuracy: 0.3750 - val_loss: 1.0894 - val_categorical_accuracy: 0.5646
Epoch 7/10
12/

# Fine-Tuning the Model

Now that we have trained our own classification layers on top of the Xception base model, we can train all layers of the model now by setting `base_model.trainable = True` (i.e. "unfreezing" the base model) and setting a very low learning rate. A low learning rate is used to prevent destroying the pre-trained its useful pre-trained features.

Doing this would ideally provide an extra boost to the model's performance, but it can lead to overfitting.

In [ ]:
# Unfreeze the base_model. Note that it keeps running in inference mode
# since we passed `training=False` when calling it. This means that
# the batchnorm layers will not update their batch statistics.
# This prevents the batchnorm layers from undoing all the training
# we've done so far.
base_model.trainable = True
model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.CategoricalAccuracy()]
)

epochs = 5
model.fit(train_generator, epochs=epochs, validation_data=validation_generator)
model.save(directory_name + "transfer_learning_classifier_fine_tuned.model")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 413, 202, 3)]     0         
                                                                 
 sequential (Sequential)     multiple                  0         
                                                                 
 rescaling (Rescaling)       (None, 413, 202, 3)       0         
                                                                 
 xception (Functional)       (None, 13, 7, 2048)       20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                             

In [ ]:
# code section adapted from https://github.com/JoshWheeler08/DolphinAcoustics-Classifier/blob/main/vip_dolphin.ipynb

classes = dict((v,k) for k,v in (train_generator.class_indices).items())

test_generator.reset()
test_labels = test_generator.classes
array_of_class_names = np.array(list(classes.values()))
y_hat = model.predict(test_generator)
print(classification_report(test_labels,y_hat.argmax(axis=1), target_names=array_of_class_names))

              precision    recall  f1-score   support

  bottlenose       0.32      0.14      0.20       200
      common       0.37      0.33      0.34       200
melon-headed       0.34      0.57      0.43       200

    accuracy                           0.35       600
   macro avg       0.34      0.35      0.32       600
weighted avg       0.34      0.35      0.32       600



In [ ]:
# code section adapted from https://github.com/JoshWheeler08/DolphinAcoustics-Classifier/blob/main/vip_dolphin.ipynb

test_generator.reset()
test_loss, test_acc = model.evaluate(test_generator, verbose=2) # TODO: Check the return values for this

# SUMMARY STATISTICS
print("----- Evaluation Summary statistics -----")
print("Test accuracy = ", test_acc)
print("Test loss = ", test_loss)

12/12 - 12s - loss: 0.8664 - categorical_accuracy: 0.6383 - 12s/epoch - 973ms/step
----- Evaluation Summary statistics -----
Test accuracy =  0.6383333206176758
Test loss =  0.8664476275444031
